# Fill In Blanks Type Questions

# **Install Libraries**

In [ ]:
# Installing from https://github.com/boudinfl/pke library for Python Keyword extraction
# We use a fixed commit as the later changes might break the code. If it was on pip we would have used exact version number.
#https://github.com/sudheernaidu53/Machine-learning-Deep-learning-projects

!pip install tika
!pip install PyPDF2
!pip install docx2txt
!pip install pdf2docx
!python -m spacy info
!pip install aspose-words
!pip install --quiet flashtext==2.7
!pip install datasets evaluate transformers[sentencepiece]
!pip install --quiet git+https://github.com/boudinfl/pke.git@5af1f817e0211c33ac3f90e1e86bb5c1283448e8

# Import Libraries

In [ ]:

import re
import pke
import tika
import nltk
import time
import json
import PyPDF2
import string
import random
import requests
import docx2txt
import traceback
import itertools
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')

import numpy as np
from time import sleep
from tika import parser
from datetime import date
from pprint import pprint
import aspose.words as aw
from itertools import chain
from datetime import datetime
from pdf2docx import Converter
from nltk import sent_tokenize
from nltk.corpus import wordnet
from multiprocessing import Pool
from requests.api import request
from transformers import pipeline
from nltk.corpus import stopwords
from collections import defaultdict
from multiprocessing import cpu_count
from flashtext import KeywordProcessor
from nltk.tokenize import sent_tokenize
from concurrent.futures import ProcessPoolExecutor
from nltk import pos_tag, word_tokenize, sent_tokenize

question_answerer = pipeline("question-answering")

In [ ]:
start = time.time()

# File Read

In [ ]:
temp_id = "new id"
request=requests.get("https://generate-questions.devbyopeneyes.com/api/getFileData/63748882333ec0fbfd0e0db2")
resp= request.json()
file_path=(resp["data"]["file_path"])
_id=(resp["data"]["_id"])
number_of_question=(resp["data"]["number_of_question"])
file_type=(resp["data"]["file_type"])

In [ ]:
if file_type == "txt":
    import urllib.request
    response = urllib.request.urlopen(file_path)
    html = response.read()
    Text=html.decode('utf8')
elif file_type=="pdf":
    url = file_path
    response = requests.get(url)
    my_raw_data = response.content

    with open("my_pdf.pdf", 'wb') as my_data:
        my_data.write(my_raw_data)

    open_pdf_file = open("my_pdf.pdf", 'rb')
    read_pdf = PyPDF2.PdfFileReader(open_pdf_file)
    if read_pdf.isEncrypted:
        read_pdf.decrypt("")
        Text = read_pdf.getPage(0).extractText()
    else:
        Text = read_pdf.getPage(0).extractText()
elif file_type == "docx":
    url = file_path
    response = requests.get(url)
    my_raw_data = response.content

    with open("my_doc.txt", "wb") as text_file:
        text_file.write(my_raw_data)

    open_docx_file = open("my_doc.txt", 'rb')

    Text = docx2txt.process(open_docx_file)
else:
    print("Invalid File Type")

In [ ]:
if Text == None or len(Text) == 0:
  print("please enter valid file")
else:
  print("valid file")

# Pipeline

In [ ]:
def Preprocessing(task1):
  sentences = sent_tokenize(task1)
  sentence_to_words=[]
  for i in sentences[0:]:
   sentence_to_words.extend(i.split())  
  tokens = [ w for w in sentence_to_words if w[0]!='[' and w[-1]!= ']' ]
  remove = string.punctuation
  remove = remove.replace(".", "").replace("-", "").replace(",", "")
  pattern = r"[{}]".format(re.escape(remove))
  table = str.maketrans('', '', pattern)
  stripped = [w.translate(table) for w in tokens]
  words_to_sentense=' '.join(stripped)
  return words_to_sentense

def tokenize_sentences(words_to_sentense):   
    sentence=[]
    sentences = sent_tokenize(words_to_sentense)
    sentences = [sentence.strip() for sentence in sentences if len(sentence) > 20]
    filtered_list_short_sentences = [sent for sent in sentences if len(sent)>30 and len(sent)<200]
    return filtered_list_short_sentences


def get_noun_adj_verb(text):   
    out=[]
    try:
        extractor = pke.unsupervised.MultipartiteRank()
        extractor.load_document(input=text,language='en')
        #    not contain punctuation marks or stopwords as candidates.
        pos = {'ADJ', 'NOUN'}
        stoplist = list(string.punctuation)
        stoplist += ['-lrb-', '-rrb-', '-lcb-', '-rcb-', '-lsb-', '-rsb-']
        stoplist += stopwords.words('english')
        # extractor.candidate_selection(pos=pos, stoplist=stoplist)
        extractor.candidate_selection(pos=pos)
        # 4. build the Multipartite graph and rank candidates using random walk,
        #    alpha controls the weight adjustment mechanism, see TopicRank for
        #    threshold/method parameters.
        extractor.candidate_weighting(alpha=1.1,
                                      threshold=0.75,
                                      method='average')
        keyphrases = extractor.get_n_best(n=30)
        for val in keyphrases:
            out.append(val[0])
    except:
        out = []
        traceback.print_exc()
    return out

def keyword(flatten_list): 
    keywords=[]
    for i in flatten_list:
        noun_verbs_adj = get_noun_adj_verb(i)
        keywords.append(noun_verbs_adj)
    return keywords

def get_sentences_for_keyword(keywords, sentences):    
    keyword_processor = KeywordProcessor()
    keyword_sentences = {}
    for word in keywords:
        keyword_sentences[word] = []
        keyword_processor.add_keyword(word)
    for sentence in sentences:
        keywords_found = keyword_processor.extract_keywords(sentence)
        for key in keywords_found:
            keyword_sentences[key].append(sentence)
    for key in keyword_sentences.keys():
        values = keyword_sentences[key]
        values = sorted(values, key=len, reverse=True)
        keyword_sentences[key] = values   
    return keyword_sentences

def keyword_sentences(keywords,flatten_list):   
    keyword_sentence=[]
    for i in keywords:
        keyword_sentence_mapping_noun_verbs_adj = get_sentences_for_keyword(i, flatten_list)
        keyword_sentence.append(keyword_sentence_mapping_noun_verbs_adj)   
    return keyword_sentence

def get_fill_in_the_blanks(sentence_mapping):   
    out={"title":"Fill in the blanks for these sentences with matching words at the top"}
    blank_sentences = []
    processed = []
    keys=[]
    generated_answer=[]
    for key in sentence_mapping:
        if len(sentence_mapping[key])>0:
            sent = sentence_mapping[key][0]
            # Compile a regular expression pattern into a regular expression object, which can be used for matching and other methods
            insensitive_sent = re.compile(re.escape(key), re.IGNORECASE)
            no_of_replacements =  len(re.findall(re.escape(key),sent,re.IGNORECASE))
            line = insensitive_sent.sub(' _________ ', sent)
            if (sentence_mapping[key][0] not in processed) and no_of_replacements<2:
                generated_answer.append({'question':line, 'context':sentence_mapping[key][0],'answer':key})
    out["sentences"]=blank_sentences[:]
    out["keys"]=keys[:]    
    return generated_answer

def fill_in_the__blank(keyword_sentence):   
    fill_in_the_blank=[]
    for i in keyword_sentence:
        fill_in_the_blanks = get_fill_in_the_blanks(i)
        fill_in_the_blank.append(fill_in_the_blanks)  
    return fill_in_the_blank


def question_context_list(fill_in_the_blank):  
    question=[]
    context=[]
    for i in fill_in_the_blank:
        for j in i:
            questions=j["question"]
            question.append(questions)
            contexts=j["context"]
            context.append(contexts) 
    return question,context

def Ranking(question_list, context_list): 
  rank=[]
  for i, j in zip(question_list, context_list):
    d = question_answerer(question=i, context=j)
    e=d.get("score")
    e_updated=1-float(e)
    accuracy_score="%.2f" % round(e_updated*100, 2)
    rank.append(accuracy_score) 
  return rank

def Fib_dict(rank,fill_in_the_blank):  
    for option_index, option in enumerate(fill_in_the_blank):      
        for i in option:
            i["rank"] = rank[option_index]
    fill_in_the_blanks = list(chain.from_iterable(fill_in_the_blank))   
    return fill_in_the_blanks


def wrong_right_blanks(fill_in_the_blank):
  wrong_blank = []
  right_blank = []
  for i in fill_in_the_blank:
      quest = i.get("question")
      if quest[0]=='_' or quest[-2]=='_':
          wrong_blank.append(i)
      else:
          right_blank.append(i)
  return  wrong_blank,right_blank

def Final_requied_blanks(wrong_blank,right_blank):
  reduced_wrong_blank = []
  for i in wrong_blank:
    ranks = (float(i['rank']) - 20.00)
    rank_updated="%.2f" % round(ranks, 2)
    if float(i['rank'])  <= 20:
      reduced_wrong_blank.append({"question": i['question'], "context": i['context'],"answer":i['answer'], "rank" : i['rank']})
    else:
      reduced_wrong_blank.append({"question": i['question'], "context": i['context'], "answer":i['answer'], "rank" : rank_updated})
  final_required_data = right_blank + reduced_wrong_blank
  return final_required_data


def Sorting(final_required_data):
  final_data = sorted(final_required_data, key=lambda i:float(i['rank']),reverse=True)
  return final_data

def flatten(list_of_lists):
    return list(itertools.chain.from_iterable(list_of_lists))

In [ ]:
def Pipeline(text):
    a=Preprocessing(Text)                         # preprocessing
    b=tokenize_sentences(a)                       # convert text to tokens
    c=keyword(b)                                  # fatch keywords
    d=keyword_sentences(c,b)                      # keywords to main sentence
    e=fill_in_the__blank(d)                       # generate blanks
    f=question_context_list(e)                    # create lists of questions and context for find rank
    g=Ranking(f[0],f[1])                          # find rank
    h=Fib_dict(g,e)                               # crate a dictionary
    i=wrong_right_blanks(h)                       # create a link of blank which have at first and last position
    j=Final_requied_blanks(i[0],i[1])             # final blanks

    return j

In [ ]:
outputs=Pipeline(Text)

x=1
Final_list=[]  
for i in outputs:
  Final_new=[]
  i['Paragraph_no']=x
  Final_new.append(i)
  x+=1
  Final_list.append(Final_new)

h= flatten(Final_list) # Flattening nested list
FIB_Questions=Sorting(h) # Our Final FIB questions with sorting

# Duplication Remove

In [ ]:
def duplicate_question(final_required_data):
    temp_arr = []
    questions_are = []
    for question in final_required_data:
        if question["question"] not in temp_arr:
            temp_arr.append(question["question"])
            questions_are.append(question)
    return questions_are

FIB=duplicate_question(FIB_Questions)

# Date & Time

In [ ]:
end = time.time()

dt1 = datetime.fromtimestamp(start)
dt2 = datetime.fromtimestamp(end)
total_time = dt2 - dt1

generated_date = date.today()

# Post Questions

In [ ]:
generate_ques=int(number_of_question)
if generate_ques > len(FIB) or generate_ques == 0:
    print("please give value between 1 to {}".format(len(FIB)))
else:
    middle_index = generate_ques
    user_required_questions = FIB[:middle_index]
    other_questions = FIB[middle_index:]
        
    for question_index, question in enumerate(user_required_questions):
      question["question_id"]= question_index +1

    for question_index, question in enumerate(other_questions):
      question["question_id"]= question_index +1

In [ ]:
url="https://generate-questions.devbyopeneyes.com/api/GenerateQuestions" 
headers = {'Content-Type':'application/json','Accept':'application/json'}
post_array ={
    "id" : _id,
    "questions" : user_required_questions,
    "other_questions" : other_questions,
    "upload_process_time": str(total_time),
    "generate_date": str(generated_date),
    "no_of_question_generate":str(len(FIB))
}
status = requests.post(url,headers=headers,data=json.dumps(post_array))